### An Attempt at Webscraping "www.monsterindia.com/jobs-in-bangalore.html" for first 40 entries or first page

##### Disclaimer: The below codes and the data generated is solely for the purpose of Education and Learning, at any point of time shall not be used for commercialization. The Author is not responsible for the outcome for such an event

##### Written by: P Akash | Linkedin: https://www.linkedin.com/in/p-akash-0b001/

#### Importing all the relevent Python Packages

In [38]:
# Importing BeautifulSoup4 to help scrape the webpage
# can be installed using "pip install beautifulsoup4"
# or for people who are using anaconda environment "conda install -c anaconda beautifulsoup4"
from bs4 import BeautifulSoup 

# Importing urllib3 to 'GET' and 'POST' request to or fro from the websites
# can be installed using "pip install urllib3"
# or for people who are using anaconda environment "conda install urllib3"
import urllib3 

#The below line will remove all the Insecure Request Warnings while Requesting a website for scraping
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Importing Pandas Package to convert all the data into DataFrames so as to easily handle the data.
# pip : "pip install pandas"
# anaconda : "conda install pandas"
import pandas as pd
from pandas import DataFrame, Series

# Importing collections package to convert the extracted data into an ordered dictionary which is further used to 
# create DataFrames
from collections import OrderedDict

# Importing time module to provide a pause in between the scraping of 2 links.
import time

#### Creating a BeautifulSoup Object or "Soup" as the community calls it, to extract data

In [39]:
# The below line is used to make requests to multiple hosts or multiple requests to the same host.
# The PoolManager will take care of Pools so that we don't have to do it manually. 
http = urllib3.PoolManager()

# We are Specifying the base URL that is what we are going to scrape
base_url = "https://www.monsterindia.com/jobs-in-bangalore-1.html"

# Requesting the website using the Base URL
response = http.request("GET",base_url)

# Creating the soup
soup = BeautifulSoup(response.data,"lxml")
#print(soup)

#### Uncomment this cell for customized scraping

In [27]:
# finding no of jobs and no of pages to scrape
# print("  No of Job Entries  ".center(110,"."))
# num_entries = int(soup.find("div",{"class" : "count pull-left"}).getText().strip()[10:-7])
# print(num_entries)
# print(type(num_entries))

# print("  No of Pages to Scrape  ".center(110,"."))
# num_of_entries_in_a_page = 40
# num_pages = num_entries // num_of_entries_in_a_page
# print(num_pages)
# print(type(num_pages))

#### Creating a new Soup to scrape first 2 pages

In [47]:
# Creating a Dataframe which will be further updated with the extracted data.
monster_df = pd.DataFrame()

# For reordering the whole DataFrame on the basis of our job, we have created a pallete of column names.
column_names = ["Job Title", 
                "Skills", 
                "Experience", 
                "Company Name", 
                "Location", 
                "Link", 
                "Job Role", 
                "Job Industry",
                "Job Description"]

# Manipulating the URL for efficient Scraping
url = "https://www.monsterindia.com/jobs-in-bangalore-"
postfix = ".html"

# We want to scrape the data for first page only, so we are going to run the for loop starting from 1 to 2.
# If we want multiple pages then we can proceed further to change the value (max 700).
for page in range(1,2):
    http = urllib3.PoolManager()    
    base_url = url + str(page) + postfix
    response = http.request("GET", base_url)
    new_soup = BeautifulSoup(response.data,"lxml")
    
    # The below line will look out for all the possible links (represented by <a></a> in HTML) which has class = "title_in"
    # and extract all the href link values and will make a list of all the links into the variable "all_links"
    all_links = [ "https:" + link.get('href') for link in new_soup.find_all('a',{"class":"title_in"}) if "job-vacancy" in str(link.get("href"))]

    # We are running a loop for all the links, such that we can scrape the data of the link in the sequential manner
    for links in all_links:
        print("\n")
        
        #Below line will print the current link in which the soup is currenlty working on
        print(links)
        http = urllib3.PoolManager()
        new_response = http.request("GET",links)
        meta_soup = BeautifulSoup(new_response.data,"lxml")
        
        # Creating a try-catch block so that the code wont interrupt at any point of time and will only end when finished
        try:
            # Extracting Company Name
            company_name = meta_soup.find("a",{"class":"company"}).getText().strip()
            print()
            
            # Extracting Job Title
            job_title = meta_soup.find("h1",{"class":"job_title_seo"}).getText().strip()
            
            # Extracting Keywords | Skills
            key_words = meta_soup.find("h2",{"keyskill skillseotag"}).getText().strip()
            keywords = key_words.split(":")
            keywords = keywords[1].split(",")
            
            # Extracting Location
            location = meta_soup.find("div",{"class":"jtxt jico ico1 pull-left"}).getText().strip()
            
            # Extracting Experience
            experience = meta_soup.find("div",{"class":"jtxt jico ico2 pull-left mar_left10"}).getText().strip()

            # Extracting Industry | Function | Role
            side_bar = meta_soup.find("div",{"class" : "col-md-3 col-xs-12 pull-right jd_rol_section"})
            side_bar_content = side_bar.getText().strip().split("\n")
            job_role = side_bar_content[1]
            #job_posted = side_bar_content[2]
            job_industry = side_bar_content[0][8:-15]
            
            # Extracting Job Description and About the Company
            about = meta_soup.find_all("div",{"class":"desc"})
            job_description = about[0].getText().strip().split("-")
            
            #Currently not scraping the information about the company, but if you are interested uncomment the below line and run it.
            #about_comp = about[1].getText()
        
        except AttributeError:
            continue
        
        # creating a temporary Ordered Dictionary out of the extracted data, so it can be easily converted to DataFrame
        df_dict = OrderedDict({"Job Title":job_title, 
                                   "Skills":keywords,
                                   "Experience":experience,
                                   "Company Name":company_name,
                                   "Location":location,
                                   "Link":links,
                                   "Job Role":job_role,
                                   "Job Industry":job_industry,
                                   "Job Description":job_description,
                                  })
        # Printing the Ordered Dictionary to see if the data is being correctly scraped or not
        print(df_dict)
        
        # Appending data to the previously made DataFrame 
        monster_df = monster_df.append(df_dict, ignore_index = True)
        
        # Checking if the DataFrames are properly created or not
        print(monster_df)
        
        # Giving a pause of 2 seconds after every link scraped,so as not to overload the System with processess
        time.sleep(2)
    print("\n")
    
    # Printing the current page no on which we are scraping the data
    print(page)

# Reordering the DataFrame according to the pallete we have created by the name "column_names"
monster_df = monster_df.reindex(columns = column_names)

# Writing DataFrame into a .csv File
monster_df.to_csv("monster_data.csv", encoding = "utf-8", index = False)



https://www.monsterindia.com/job-vacancy-walkin-interview-in-chennai-for-bo-engineer-in-an-mnc-company-quess-corp-limited-bengaluru-bangalore-5-10-years-23273704.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'India   IndustryTelecom FunctionTelecom/ ISP Role'), ('Job Title', 'Walkin interview in chennai for BO Engineer in an MNC company'), ('Job Industry', 'Details:Venue Details: Shall be available on successful application.City: ChennaiDate: 19/01/2019 To 19/01/2019Timings: 9:30 AM to 6:30'), ('Company Name', 'Quess Corp Limited'), ('Experience', '5 - 10 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-walkin-interview-in-chennai-for-bo-engineer-in-an-mnc-company-quess-corp-limited-bengaluru-bangalore-5-10-years-23273704.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Ski



https://www.monsterindia.com/job-vacancy-email-support-web-chart-process-back-end-process-non-voice-call-center-bpo-semi-voice-inspiration-manpower-consultancy-private-limited-bengaluru-bangalore-0-5-years-23273686.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'India   Industry • Banking/ Financial Services •  Insurance •  ITES/BPO •  IT/ Computers - Hardware •  IT/ Computers - Software •  KPO/Analytics •  Recruitment/Staffing/RPO •  Telecom •  Consultancy •  Medical Transcription •  NGO/Social Services •  Social Media •  Travel/ Tourism •  Any Function • Customer Service/ Call Centre/ BPO • Telecom/ ISP Role'), ('Job Title', 'Email Support / Web Chart Process / Back end Process / Non Voice / Call Center / BPO / Semi Voice'), ('Job Industry', ''), ('Company Name', 'Inspiration Manpower Consultancy Private Limited'), ('Experience', '0 - 5 years'), ('Link', 'https://www.mon



https://www.monsterindia.com/job-vacancy-java-lead-developer-black-white-business-solutions-bengaluru-bangalore-6-11-years-23273669.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Software Engineer/ Programmer '), ('Job Title', 'Java Lead Developer'), ('Job Industry', 'IT/ Computers - Software '), ('Company Name', 'Black & White Business Solutions'), ('Experience', '6 - 11 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-java-lead-developer-black-white-business-solutions-bengaluru-bangalore-6-11-years-23273669.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' java', ' Web Services', ' AWS', ' AZURE']), ('Job Description', ['Dear Candidate,Greetings!!I am Mailing you regarding job opening for Java Lead\xa0 Developer with Siemens TechnologyYears of Experience', ' 6'



https://www.monsterindia.com/job-vacancy-linux-device-driver-development-black-white-business-solutions-bengaluru-bangalore-noida-3-9-years-23273642.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Software Engineer/ Programmer Education'), ('Job Title', 'Linux Device Driver Development'), ('Job Industry', 'IT/ Computers - Software '), ('Company Name', 'Black & White Business Solutions'), ('Experience', '3 - 9 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-linux-device-driver-development-black-white-business-solutions-bengaluru-bangalore-noida-3-9-years-23273642.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' "Linux Device Driver"', ' "Device Driver"', ' ARM', ' RTOS ', ' Linux Kernel', ' Microcontroller', ' Kernel Programming', ' Linux Internals ', ' System pr



https://www.monsterindia.com/job-vacancy-immideat-openings-in-mnc-quess-corp-limited-bengaluru-bangalore-0-5-years-23273613.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', ' • Customer Service Executive (Voice) • Customer Service Executive (Non-voice) • Technical Support Executive ( voice) • Fresher Education'), ('Job Title', 'Immideat openings in MNC'), ('Job Industry', ' • ITES/BPO •  Telecom FunctionCustomer Service/ Call C'), ('Company Name', 'Quess Corp Limited'), ('Experience', '0 - 5 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-immideat-openings-in-mnc-quess-corp-limited-bengaluru-bangalore-0-5-years-23273613.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' Fresher', ' Customer Support', ' Customer Service', ' Customer Support Executive', ' Customer Car

...



https://www.monsterindia.com/job-vacancy-sales-executive-multi-recruit-bengaluru-bangalore-2-4-years-22955342.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Sales Exec/ Sales Representative Education'), ('Job Title', 'Sales Executive'), ('Job Industry', 'Printing/ Packaging FunctionSales/ Business D'), ('Company Name', 'Multi Recruit'), ('Experience', '2 - 4 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-sales-executive-multi-recruit-bengaluru-bangalore-2-4-years-22955342.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' Sales', ' Sales Officer', ' Packaging Sales', ' Printing Sales', ' Commercial Printing Sales', ' Sales Executive', ' Business Development', ' BDE']), ('Job Description', ['Achieve revenue targets within operating budgets.\xa0Perform Prospecting



https://www.monsterindia.com/job-vacancy-senior-hardware-embedded-engineer-multi-recruit-bengaluru-bangalore-6-7-years-22955829.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Hardware Design Engineer Education'), ('Job Title', 'Senior Hardware Embedded Engineer'), ('Job Industry', 'IT/ Computers - Hardware '), ('Company Name', 'Multi Recruit'), ('Experience', '6 - 7 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-senior-hardware-embedded-engineer-multi-recruit-bengaluru-bangalore-6-7-years-22955829.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' EMI', ' AMC', ' BLDC & PMSM', ' Power Electronics', ' Embedded Hardware', ' Hardware Embedded Engineer', ' Embedded Engineer']), ('Job Description', ['Must be a expert in Power electronics\xa0B.Tech and M.Tech in Power



https://www.monsterindia.com/job-vacancy-mis-executive-multi-recruit-bengaluru-bangalore-1-3-years-22973145.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Other Roles Education'), ('Job Title', 'MIS Executive'), ('Job Industry', 'IT/ Computers '), ('Company Name', 'Multi Recruit'), ('Experience', '1 - 3 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-mis-executive-multi-recruit-bengaluru-bangalore-1-3-years-22973145.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' MS OFFICE', ' Excel', ' PowerPoint', ' Data Handling']), ('Job Description', ['Knowledge in Excel/ Advance excel/ Data handling\xa0Good communication skills and Excellent team player.\xa0Always having attitude towards work completion with dedication\xa0Good Knowledge on MS OFFICE, Excel, PowerPoint\xa



https://www.monsterindia.com/job-vacancy-c-developer-multi-recruit-bengaluru-bangalore-4-6-years-22870744.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords=

OrderedDict([('Job Role', 'Software Engineer/ Programmer Education'), ('Job Title', 'C# Developer'), ('Job Industry', 'IT/ Computers - Software '), ('Company Name', 'Multi Recruit'), ('Experience', '4 - 6 years'), ('Link', 'https://www.monsterindia.com/job-vacancy-c-developer-multi-recruit-bengaluru-bangalore-4-6-years-22870744.html?sig=js-1--1&from=&rfr=refine;loc=2;lmy=1;day=365;ref=%2F%2Fwww%2Emonsterindia%2Ecom%2Fjob%2Dsearch%2Ehtml;res_cnt=4%30&hlWords='), ('Skills', [' C#', ' Webservices-REST', ' REST', ' SOAP', ' WSDL', ' C# Developer']), ('Job Description', ['Proficient in C#, with a good knowledge of its ecosystems.\xa0Familiarity with the .NET framework (you may specify particular versions based on your requirements).\xa0Familiarity with 



1


In [48]:
# Printing DataFrame
monster_df

,Job Title,Skills,Experience,Company Name,Location,Link,Job Role,Job Industry,Job Description
0,Walkin interview in chennai for BO Engineer in...,"[ Firewall, Fortigate, Fortinet, Arbor, Lo...",5 - 10 years,Quess Corp Limited,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-walki...,India IndustryTelecom FunctionTelecom/ ISP Role,Details:Venue Details: Shall be available on s...,[Profile : IP Security Engineer.experience : 5...
1,Software Engineer -Java Fullstack,"[ Java, J2EE, Spring, Hibernate, anguler2 ...",3 - 10 years,Black & White Business Solutions,"Bengaluru / Bangalore, Hyderabad / Secunderabad",https://www.monsterindia.com/job-vacancy-softw...,India IndustryIT/ Computers - Software Funct...,,[Should have experience with java and angular ...
2,Job Opening for Hitachi & EMC Storage,"[ Hitachi Storage, EMC]",6 - 11 years,Black & White Business Solutions,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-job-o...,Software Engineer/ Programmer,IT/ Computers - Software,"[Dear Candidate,Greetings!!I am Mailing you re..."
3,Email Support / Web Chart Process / Back end P...,"[ Nonvoice, BPO, Backend, Processing, Voic...",0 - 5 years,Inspiration Manpower Consultancy Private Limited,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-email...,India Industry • Banking/ Financial Services...,,[Greetings of the Day!!!!!! We have Immediate ...
4,Manager Varistor Education,"[ B2C Sales, Educational Sales, Marketing, ...",0 - 0 years,Varistor Technologies Private limited,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-manag...,India IndustryEducation FunctionSales/ Busin...,,"[, End to end job which includes a. Explai..."
5,Java Lead Developer,"[ java, Web Services, AWS, AZURE]",6 - 11 years,Black & White Business Solutions,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-java-...,Software Engineer/ Programmer,IT/ Computers - Software,"[Dear Candidate,Greetings!!I am Mailing you re..."
6,Hiring for International Voice support with sa...,"[ bpo, voice process, inbound, outbound,...",1 - 4 years,Gettjobs Recruitment Firm,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-hirin...,• Customer Service Executive (Voice) • Techni...,ITES/BPO FunctionCustomer Service/ Call C,[Greetings from Gett jobs!Excellent Hiring for...
7,Linux Device Driver Development,"[ ""Linux Device Driver"", ""Device Driver"", AR...",3 - 9 years,Black & White Business Solutions,"Bengaluru / Bangalore, Noida",https://www.monsterindia.com/job-vacancy-linux...,Software Engineer/ Programmer Education,IT/ Computers - Software,"[Job Description· UART and SPI, I2C, Embedded ..."
8,SFDC Technical Lead,"[ SFDC Developer"", ""Salesforce Development"", ...",2 - 10 years,Acs Global Tech Solutions Private Limited,"Bengaluru / Bangalore, Hyderabad / Secunderabad",https://www.monsterindia.com/job-vacancy-sfdc-...,Software Engineer/ Programmer,IT/ Computers - Software,[Salesforce Technical LeadWork you’ll do· Desi...
9,Immideat openings in MNC,"[ Fresher, Customer Support, Customer Servic...",0 - 5 years,Quess Corp Limited,Bengaluru / Bangalore,https://www.monsterindia.com/job-vacancy-immid...,• Customer Service Executive (Voice) • Custom...,• ITES/BPO • Telecom FunctionCustomer Servic...,"[Hi,My name is Sharath, Associate HR at IKYA ..."
